In [6]:
import pandas as pd
import numpy as np
import requests
import json
import time
from scipy.optimize import linear_sum_assignment

In [7]:
bus_df=pd.read_csv('nps routes.csv')
bus_df.head()

,Sl No,College/Institute,Vehicle Number,Route Number,Driver Point Latitude,Driver Point Longitude,Driver Point Name,First Pickup Pt. Latitude,First Pickup Pt. Longitude,First Pickup Point Name
0,487,NPSWF,KA07B1355,1,12.952958,77.804186,Muthkur,12.938821,77.721085,"Balgere, Sobha Dream Acres Apt"
1,488,NPSWF,KA53D8480,2,12.952708,77.726000,Siddapura,12.900606,77.747839,"Gunjur, Greenwood High School"
2,489,NPSWF,KA53D8723,3,12.961514,77.615152,Austin Town,12.974034,77.682899,"Vignanagar, Jain Heights East Parade Apt"
3,490,NPSWF,KA53D8726,4,13.012996,77.698353,K R Puram Extn,12.985157,77.674090,"Kaggadaspura main road, Airtel office"
4,491,NPSWF,KA53AB9107,5,13.017325,77.657404,Baghirathi Shed,13.013863,77.664028,"Rm nagar, Nellore mess"


In [1]:
cluster_name=input('Name of the current Cluster: ')

Name of the current Cluster:  NPS


In [139]:
driver_df=bus_df[['Vehicle Number', 'Route Number', 'Driver Point Latitude', 'Driver Point Longitude', 'Driver Point Name', 'College/Institute']].rename(columns={
    'Vehicle Number':'bus', 
    'Route Number': 'route',
    'Driver Point Latitude': 'dlat',
    'Driver Point Longitude': 'dlon',
    'Driver Point Name': 'dname',
    'College/Institute': 'cname'
})
pickup_df=bus_df[['Vehicle Number', 'Route Number', 'First Pickup Pt. Latitude', 'First Pickup Pt. Longitude', 'First Pickup Point Name', 'College/Institute']].rename(columns={
    'Vehicle Number':'bus', 
    'Route Number': 'route',
    'First Pickup Pt. Latitude': 'plat',
    'First Pickup Pt. Longitude': 'plon',
    'First Pickup Point Name': 'pname',
    'College/Institute': 'cname'
})

In [107]:
driver_df.head()
driver_df.set_index('bus')

,route,dlat,dlon,dname
bus,,,,
KA07B1355,1,12.952958,77.804186,Muthkur
KA53D8480,2,12.952708,77.726000,Siddapura
KA53D8723,3,12.961514,77.615152,Austin Town
KA53D8726,4,13.012996,77.698353,K R Puram Extn
KA53AB9107,5,13.017325,77.657404,Baghirathi Shed
...,...,...,...,...
KA53AB0458,9,13.003911,77.748979,NPS Parking
KA53AB0965,10,12.977053,77.752556,Whitefield Vijayanagar
KA53AB0465,11,13.011130,77.780754,Dps Parking


In [106]:
pickup_df.head()
pickup_df.set_index('bus')

,route,plat,plon,pname
bus,,,,
KA07B1355,1,12.938821,77.721085,"Balgere, Sobha Dream Acres Apt"
KA53D8480,2,12.900606,77.747839,"Gunjur, Greenwood High School"
KA53D8723,3,12.974034,77.682899,"Vignanagar, Jain Heights East Parade Apt"
KA53D8726,4,12.985157,77.674090,"Kaggadaspura main road, Airtel office"
KA53AB9107,5,13.013863,77.664028,"Rm nagar, Nellore mess"
...,...,...,...,...
KA53AB0458,9,13.062468,77.741392,"Bommenahalli, Prestige Tranquility Apt"
KA53AB0965,10,12.964907,77.757644,"Hagdur main road, GOYAL ORCHID WHITEFIELD Apt"
KA53AB0465,11,12.990480,77.789359,"Soukya Road, Birla Alokya Whitefield Apt"


In [12]:
base_url= "http://localhost:5000/route/v1/driving"

In [13]:
def osrm_calculate(lat1, lon1, lat2, lon2): #input parameters as default lat,lon
    src=f'{lon1},{lat1};{lon2},{lat2}'
    curl=f"http://localhost:5000/route/v1/driving/{src}?overview=false"
    time.sleep(0.2)
    try:
        json_response=requests.get(curl).json()
        return json_response['routes'][0]['distance']/1000
    except Exception as exc:
        print(f"OSRM error at {lat1},{lon1} to {lat2},{lon2}: {exc}")
        return np.inf

In [94]:
buses=bus_df['Vehicle Number'].values
buses

array(['KA07B1355', 'KA53D8480', 'KA53D8723', 'KA53D8726', 'KA53AB9107',
       'KA53D8404', 'KA53D8473', 'KA53D8547', 'KA53D8727', 'KA07B1354',
       'KA53D8471', 'KA53D8405', 'KA53D8403', 'KA53D8722', 'KA53AB9114',
       'KA51AJ8288', 'KA53D8543', 'KA53D8472', 'KA53D8728', 'KA53D5167',
       'KA53D8720', 'KA53D8540', 'KA53D8537', 'KA53D8402', 'KA53D8399',
       'KA53D8549', 'KA53D2722', 'KA53D8550', 'KA53D4604', 'KA53D2898',
       'KA53D8551', 'KA53D8729', 'KA53D8541', 'KA53D5164', 'KA53D8401',
       'KA53D2720', 'KA53D8539', 'KA53D8542', 'KA53D4605', 'KA53D2957',
       'KA53D2899', 'KA53AB9116', 'KA53D8400', 'KA53D8477', 'KA53D2897',
       'KA53D2723', 'KA53D5163', 'KA53D8475', 'KA53D8545', 'KA53AB9122',
       'KA53D8548', 'KA53D8476', 'KA53D8470', 'KA53D8724', 'KA53D8474',
       'KA53D8544', 'KA53D8546', 'KA53AB0464', 'KA53AB0969', 'KA53AB0467',
       'KA53AB0460', 'KA53AB0468', 'KA53AB0966', 'KA53AB0466',
       'KA53AB0462', 'KA53AB0458', 'KA53AB0965', 'KA53AB0465',
  

In [10]:
for bus in buses:
    for pickup in buses:
        distance=osrm_calculate(bus_coords[bus]['dlat'], bus_coords[bus]['dlon'], pickup_coords[pickup]['plat'], pickup_coords[pickup]['plon'])
        distance_mat.loc[bus, pickup] =distance 
distance_mat

,KA07B1355,KA53D8480,KA53D8723,KA53D8726,KA53AB9107,KA53D8404,KA53D8473,KA53D8547,KA53D8727,KA07B1354,...,KA53AB0460,KA53AB0468,KA53AB0966,KA53AB0466,KA53AB0462,KA53AB0458,KA53AB0965,KA53AB0465,KA53AB0463,KA53AB0967
KA07B1355,14.3195,16.8767,21.8954,20.6325,22.6640,23.1258,10.9148,20.6219,20.1865,18.5033,...,23.1258,14.2794,20.6325,19.9723,20.7517,23.1562,7.0697,7.0414,6.4003,15.3478
KA53D8480,3.7339,10.6795,7.8200,9.7516,13.2416,13.7034,25.3391,6.5464,6.1110,4.4278,...,13.7034,6.7839,9.7516,14.4583,11.3294,16.8022,4.5355,10.2144,6.5593,1.2724
KA53D8723,14.7693,23.0239,10.7253,8.9001,10.1991,10.6609,37.6835,13.6796,11.2889,13.4352,...,10.6609,13.8851,8.9001,20.4669,7.9645,20.6194,16.8799,22.5588,18.9037,12.8421
KA53D8726,13.5361,19.7961,9.1808,6.9452,6.8167,6.1194,28.2280,7.5040,7.2483,7.8617,...,6.1194,5.2060,6.9452,8.7985,4.9044,8.9510,11.5993,13.1033,12.2620,11.6090
KA53AB9107,18.0844,24.4723,8.8487,8.6263,1.7180,2.1798,32.9043,9.4643,9.2086,10.1411,...,2.1798,9.8822,8.6263,14.9597,3.9616,15.1121,16.2755,17.7795,16.9382,16.1573
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
KA53AB0458,12.3722,14.7134,11.8783,10.9289,12.6423,11.9451,21.8132,9.9384,9.6827,7.8458,...,11.9451,5.1900,10.9289,7.4652,10.7301,9.8090,6.5165,6.6884,5.8471,9.9107
KA53AB0965,8.0505,10.3917,12.1366,12.3022,14.3337,14.7955,20.3428,10.8631,10.4276,8.7445,...,14.7955,5.9491,12.3022,9.4620,12.4215,11.8059,2.1948,5.2181,2.7289,5.5890
KA53AB0465,15.2881,17.6293,19.3742,18.1112,25.6733,24.9761,19.6449,18.1007,17.6652,15.9821,...,24.9761,11.7581,18.1112,11.4183,23.7611,14.6023,8.0077,4.0529,6.5907,12.8266
KA53AB0463,12.3722,14.7134,11.8783,10.9289,12.6423,11.9451,21.8132,9.9384,9.6827,7.8458,...,11.9451,5.1900,10.9289,7.4652,10.7301,9.8090,6.5165,6.6884,5.8471,9.9107


In [4]:
dist_mat_name=input('Distance matrix name: ')

Distance matrix name:  NPS_dist_matrix


In [23]:
distance_mat=pd.read_csv(f'{dist_mat_name}.csv')

In [17]:
distance_mat.to_csv('NPS_dist_matrix.csv', index=False)

NameError: name 'distance_mat' is not defined

In [125]:
optim_drivers, optim_pickups=linear_sum_assignment(distance_mat)
driver = distance_mat.index[optim_drivers]
pickup = distance_mat.index[optim_pickups]

In [124]:
 driver

Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
       36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53,
       54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69],
      dtype='int64')

In [158]:
optimized_df = pd.DataFrame({
    'From Bus': distance_mat.columns[optim_drivers],
    'Driver Site':driver_df.loc[driver,'cname'].values,
    'Driver pt lat': driver_df.loc[driver,'dlat'].values,
    'Driver pt long': driver_df.loc[driver,'dlon'].values,
    'Driver pt name': driver_df.loc[driver,'dname'].values,
    'Driver Route': driver_df.loc[driver,'route'].values,
    'To Bus': distance_mat.columns[optim_pickups],
    'Pickup Route': pickup_df.loc[pickup,'route'].values,
    'Pickup pt name': pickup_df.loc[pickup,'pname'].values,
    'Pickup pt lat': pickup_df.loc[pickup,'plat'].values,
    'Pickup pt long': pickup_df.loc[pickup,'plon'].values,
    'Pickup Site':pickup_df.loc[pickup,'cname'].values,
    'Optimized dead km': distance_mat.values[optim_drivers, optim_pickups],
    'Original dead km': distance_mat.values[optim_drivers, optim_drivers]
})

In [159]:
optimized_df

,From Bus,Driver Site,Driver pt lat,Driver pt long,Driver pt name,Driver Route,To Bus,Pickup Route,Pickup pt name,Pickup pt lat,Pickup pt long,Pickup Site,Optimized dead km,Original dead km
0,KA07B1355,NPSWF,12.952958,77.804186,Muthkur,1,KA53D8545,50,"Immadehalli main road, Kavery Bakery",12.967186,77.760430,NPSWF,6.5891,14.3195
1,KA53D8480,NPSWF,12.952708,77.726000,Siddapura,2,KA53D8402,25,"Aecs Layout 60 feet road, BVL Serene Apt",12.958672,77.714163,NPSWF,1.9190,10.6795
2,KA53D8723,NPSWF,12.961514,77.615152,Austin Town,3,KA53D8470,T2,"Mallesh palya, More store",12.974034,77.672986,NPSWF,9.3471,10.7253
3,KA53D8726,NPSWF,13.012996,77.698353,K R Puram Extn,4,KA53AB0460,4,"Rm nagar, D mart",13.012072,77.667980,NPSPP,6.1194,6.9452
4,KA53AB9107,NPSWF,13.017325,77.657404,Baghirathi Shed,5,KA53AB9107,5,"Rm nagar, Nellore mess",13.013863,77.664028,NPSWF,1.7180,1.7180
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,KA53AB0458,NPSPP,13.003911,77.748979,NPS Parking,9,KA53D8723,3,"Vignanagar, Jain Heights East Parade Apt",12.974034,77.682899,NPSWF,11.8783,9.8090
66,KA53AB0965,NPSPP,12.977053,77.752556,Whitefield Vijayanagar,10,KA53AB0967,13,"Thubarahalli, Ukn Esperanza Apt / Drop Klay wh...",12.952514,77.720706,NPSPP,5.5890,2.1948
67,KA53AB0465,NPSPP,13.011130,77.780754,Dps Parking,11,KA53D2723,47,"Hagdur main road, GOYAL ORCHID WHITEFIELD Apt",12.964907,77.757644,NPSWF,8.0077,4.0529
68,KA53AB0463,NPSPP,13.003911,77.748979,NPS Parking,12,KA53AB9122,51,"Seegehalli-kannamangala road, Urbanx Earth N Sky",13.017312,77.767853,NPSWF,3.5878,5.8471


In [144]:
optimized_df.to_excel(f'{cluster_name}_Optimal.xlsx', index=False)

In [148]:
def find_swap_chains(from_buses, to_buses):
    assignments = dict(zip(from_buses, to_buses))
    seen = set()
    chains = []

    for start in from_buses:
        if start not in seen:
            chain = []
            current = start
            while current not in seen and current in assignments:
                chain.append(current)
                seen.add(current)
                current = assignments[current]
                if current == chain[0]:  # Closed loop
                    chain.append(current)
                    break
            if len(chain) > 1:
                chains.append(chain)
    return chains


In [155]:
chains = find_swap_chains(optimized_df['From Bus'], optimized_df['To Bus'])

for i, chain in enumerate(chains, 1):
    print(f"Swap Chain {i}:", " → ".join(chain))


Swap Chain 1: KA07B1355 → KA53D8545 → KA07B1355
Swap Chain 2: KA53D8480 → KA53D8402 → KA53D5167 → KA53D8405 → KA53AB0965 → KA53AB0967 → KA53D8476 → KA53D8480
Swap Chain 3: KA53D8723 → KA53D8470 → KA53D8729 → KA53D8537 → KA53D5163 → KA53D8539 → KA53AB0466 → KA53D2720 → KA53D5164 → KA53D8474 → KA53AB0468 → KA53AB9116 → KA53AB0966 → KA53D8728 → KA53D8472 → KA53D4605 → KA53D8471 → KA53D8547 → KA53D2897 → KA07B1354 → KA53AB0462 → KA53D8726 → KA53AB0460 → KA53D8399 → KA53D8551 → KA53D2722 → KA53D8722 → KA53D8400 → KA53D8724 → KA53D8541 → KA53D2898 → KA53D8727 → KA53D8550 → KA53AB0464 → KA53D4604 → KA53AB0465 → KA53D2723 → KA53D8403 → KA53D8548 → KA53D8475 → KA53AB0463 → KA53AB9122 → KA53D8720 → KA53AB0467 → KA53D8544 → KA53D8542 → KA51AJ8288 → KA53AB9114 → KA53D2899 → KA53D8543 → KA53D8477 → KA53AB0969 → KA53D8546 → KA53D8401 → KA53AB0458 → KA53D8723
Swap Chain 4: KA53AB9107 → KA53AB9107
Swap Chain 5: KA53D8404 → KA53D8404
Swap Chain 6: KA53D8473 → KA53D8473
Swap Chain 7: KA53D8540 → KA53D85

In [156]:
chain_data = []
cleaned_chains = [[bus.strip() for bus in chain] for chain in chains]

for i, chain in enumerate(cleaned_chains, 1):
    route_seq = [str(driver_routes.get(bus, 'NA')) for bus in chain]
    chain_data.append({
        'Swap Chain': f'Chain {i}',
        'Routes': route_seq,
        'Vehicles': chain
    })
    
chains_df = pd.DataFrame(chain_data)


In [157]:
chains_df

,Swap Chain,Routes,Vehicles
0,Chain 1,"[1, 50, 1]","[KA07B1355, KA53D8545, KA07B1355]"
1,Chain 2,"[2, 25, 21, 13, 10, 13, T1, 2]","[KA53D8480, KA53D8402, KA53D5167, KA53D8405, K..."
2,Chain 3,"[3, T2, 33, 24, 48, 38, 7, 37, 35, T4, 5, 43, ...","[KA53D8723, KA53D8470, KA53D8729, KA53D8537, K..."
3,Chain 4,"[5, 5]","[KA53AB9107, KA53AB9107]"
4,Chain 5,"[6, 6]","[KA53D8404, KA53D8404]"
5,Chain 6,"[7, 7]","[KA53D8473, KA53D8473]"
6,Chain 7,"[23, 23]","[KA53D8540, KA53D8540]"
7,Chain 8,"[27, 27]","[KA53D8549, KA53D8549]"
8,Chain 9,"[41, 41]","[KA53D2957, KA53D2957]"


In [44]:
assert assignments['From_Bus'].is_unique, "Duplicate drivers!"
assert assignments['To_Bus'].is_unique, "Duplicate pickups!"
